In [2]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import os
from imblearn.over_sampling import SMOTE
from sample import create_sampler
from tree_extractor import path_extractor
from lightgbm import LGBMClassifier

random_state = 10

class ExpModel:
    def __init__(self, dataset, model):
        self.dataset = dataset
        self.model = model
        self.n_splits = 4
        self._accuracy = []
        self._precision = []
        self._recall = []
        self._f1_score = []

    def init(self):
        if self.model == 'lightgbm':
            if self.dataset == 'bankruptcy':
                self.target = 'Bankrupt?'
                parameters = {
                    'n_estimators': 400,
                    'learning_rate': 0.25,
                    'num_leaves': 250,
                    'max_depth': 11,
                    'min_data_in_leaf': 320,
                    'lambda_l1': 0.65,
                    'lambda_l2': 1.08,
                    'bagging_fraction': 0.71,
                    'bagging_freq': 8,
                    'feature_fraction': 0.27,
                }

                data_table = pd.read_csv('data/bank.csv')
                X = data_table.drop(self.target, axis=1).values
                y = data_table[self.target].values
            self.data_table = data_table
            self.X = X
            self.y = y
            self.parameters = parameters
            
            kf = KFold(n_splits = self.n_splits, random_state=random_state, shuffle=True)
            self.splits = []
            for train_index, test_index in kf.split(X):
                self.splits.append((train_index, test_index))
            self.fold = 0

    def has_next_fold(self):
        return self.fold < len(self.splits)
    
    def next_fold(self):
        self.fold += 1

    def train(self):
        sm = SMOTE(random_state=random_state)
        data_table = self.data_table
        X = self.X
        y = self.y
        parameters = self.parameters
        train_index, test_index = self.splits[self.fold]
        X_train = self.X[train_index]
        y_train = self.y[train_index]
        X_test = self.X[test_index]
        y_test = self.y[test_index]
        X_train, y_train = sm.fit_resample(X_train, y_train)
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.data_table = data_table
        if self.model == 'RF':
            clf = RandomForestClassifier(**parameters)
        else:
            clf = LGBMClassifier(**parameters)
        clf.fit(X_train, y_train)
        self.clf = clf
        self.y_pred = clf.predict(X_test)
        self.features = data_table.drop(self.target, axis=1).columns.to_list()

    def evaluate(self):
        _accuracy = accuracy_score(self.y_test, self.y_pred)
        _precision = precision_score(self.y_test, self.y_pred)
        _recall = recall_score(self.y_test, self.y_pred)
        _f1_score = f1_score(self.y_test, self.y_pred)
        print('Accuracy Score is', _accuracy)
        print('Precision is', _precision)
        print('Recall is', _recall)
        print('F1 Score is', _f1_score)
        self._accuracy.append(_accuracy)
        self._precision.append(_precision)
        self._recall.append(_recall)
        self._f1_score.append(_f1_score)
    
    def summary(self):
        return float(np.mean(self._accuracy)), float(np.mean(self._precision)), float(np.mean(self._recall)), float(np.mean(self._f1_score))

    def oversampling(self, rate = 2):
        is_continuous = []
        is_categorical = []
        is_integer = []

        for feature in self.data_table.columns:
            if feature == self.target:
                continue
            if self.data_table[feature].dtype == 'O':
                is_continuous.append(False)
                is_categorical.append(True)
            else:
                is_continuous.append(True)
                is_categorical.append(False)
            is_integer.append(False)
        sampler = create_sampler(self.X_train, is_continuous, is_categorical, is_integer)
        return sampler(len(self.X_train) * rate)

    def generate_paths(self):
        if self.model == 'RF':
            paths = path_extractor(self.clf, 'random forest', (self.X_train, self.y_train))
        else:
            paths = path_extractor(self.clf, 'lightgbm')
        print('num of paths', len(paths))
        return paths


In [3]:
dataset = 'bankruptcy'
model = 'lightgbm'
exp = ExpModel(dataset, model)
exp.init()
exp.train()
paths = exp.generate_paths()
exp.evaluate()


[LightGBM] [Warning] lambda_l1 is set=0.65, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.65
[LightGBM] [Warning] bagging_fraction is set=0.71, subsample=1.0 will be ignored. Current value: bagging_fraction=0.71
[LightGBM] [Warning] lambda_l2 is set=1.08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.08
[LightGBM] [Warning] feature_fraction is set=0.27, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.27
[LightGBM] [Warning] min_data_in_leaf is set=320, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=320
[LightGBM] [Warning] bagging_freq is set=8, subsample_freq=0 will be ignored. Current value: bagging_freq=8
num of paths 4111
Accuracy Score is 0.9747800586510263
Precision is 0.6440677966101694
Recall is 0.6333333333333333
F1 Score is 0.6386554621848739


In [9]:
print(len(paths))

3762


In [5]:
from tree_extractor import assign_samples_lgbm as assign_samples
assign_samples(paths, (exp.X, exp.y))
paths = [p for p in paths if p['coverage'] > 0]

In [10]:
last_paths = paths
name2path = {}
for index, path in enumerate(paths):
    name2path[path['name']] = path
    path['level'] = 0

params = [600, 80]
level_info = {}

from model_extractor_maxnum import Extractor
for level, n in enumerate(params):
    tau = (n / 80) ** 0.5
    ex = Extractor(last_paths, exp.X_train, exp.clf.predict(exp.X_train))
    w, _, fidelity_train = ex.extract(n, tau)
    [idx] = np.nonzero(w)

    accuracy_train = ex.evaluate(w, exp.X_train, exp.y_train)
    accuracy_test = ex.evaluate(w, exp.X_test, exp.y_test)
    fidelity_test = ex.evaluate(w, exp.X_test, exp.clf.predict(exp.X_test))
    print(level, n, 'accuracy_train', accuracy_train, 'accuracy_test', accuracy_test, 'fidelity_test', fidelity_test)
    
    level_info[level + 1] = {
        'fidelity_test': fidelity_test,
        'accuracy_test': accuracy_test,
    }
    for i in idx:
        name2path[last_paths[i]['name']]['level'] = level + 1
    curr_paths = [last_paths[i] for i in idx]
    last_paths = curr_paths
    print(len(last_paths))

total path weight:  0.999999999999997
0 600 accuracy_train 0.9995962858296327 accuracy_test 0.9695014662756598 fidelity_test 0.987683284457478
600
total path weight:  0.9999999999999992
1 80 accuracy_train 0.9757771497779572 accuracy_test 0.9395894428152493 fidelity_test 0.950733137829912
80


In [11]:
import shap

explainer = shap.Explainer(exp.clf)
shap_values = explainer(exp.X)
shap_values = shap_values[:,:,0]
new_shaps = []

new_feature = {}
features = [feature for feature in exp.data_table.columns if feature != exp.target]
for index, feature in enumerate(features):
    if ' - ' in feature:
        name, p = feature.split(' - ')
        p = int(p)
        if name not in new_feature:
            new_feature[name] = []
        while p >= len(new_feature[name]):
            new_feature[name].append(-1)
        new_feature[name][p] = index
    else:
        new_feature[feature] = [index]

In [12]:
features = []
feature_index = {}
feature_type = {}
for key in new_feature:
    if len(new_feature[key]) == 1:
        i = new_feature[key][0]
        features.append({
            "name": key,
            "range": [min(exp.data_table[key].values), max(exp.data_table[key].values)],
            "importance": exp.clf.feature_importances_[i],
            "dtype": "numeric",
        })
        shaps = shap_values[:, i]
        feature_index[i] = len(features) - 1
        feature_type[i] = "numeric"
    else:
        features.append({
            "name": key,
            "range": [0, len(new_feature[key])],
            "importance": sum([exp.clf.feature_importances_[i] for i in new_feature[key] if i != -1]),
            "dtype": "categoric",
        })
        feature_idxs = [i for i in  new_feature[key] if i != -1]
        shaps = shap_values[:, feature_idxs[0]]
        for i in feature_idxs[1:]:
            shaps = shaps + shap_values[:, i]

        for index, i in enumerate(new_feature[key]):
            if i != -1:
                feature_index[i] = [len(features) - 1, index]
                feature_type[i] = "categoric"
    new_shaps.append(shaps)

In [22]:
for path in paths:
    new_range = {}
    for index in path['range']:
        if feature_type[int(index)] == 'numeric':
            i = feature_index[int(index)]
            new_range[i] = path['range'][index]
        else:
            i, j = feature_index[int(index)]
            if i not in new_range:
                new_range[i] = [0] * features[i]['range'][1]
                if path['range'][index][0] <= 1 and 1 <= path['range'][index][1]:
                    new_range[i][j] = 1
                else:
                    for k in range(len(new_range[i])):
                        if k != j:
                            new_range[i][k] = 1
                        new_range[i][j] = 0
    path['range'] = new_range
    path['represent'] = False

for i in idx:
    paths[i]['represent'] = True

output_data = {
    'paths': paths,
    'features': features,
    'selected': [paths[i]['name'] for i in idx],
    'shap_values': new_shaps,
    'model_info': {
        'accuracy': exp._accuracy[-1],
        'info': level_info,
        'num_of_rules': len(paths),
        'dataset': 'Taiwan Company Bankruptcy',
        'model': 'LightGBM',
    }
}

In [24]:
import pickle
pickle.dump(output_data, open('output/bankruptcy0117.pkl', 'wb'))